In [1]:
import torch,torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
PROJECT_NAME = 'Car-Object-Detection-V11-Learning-Detectron2-V2'

In [7]:
data = pd.read_csv('./data.csv')

In [8]:
len(data)

559

In [9]:
info = data.iloc[185]

In [10]:
data['xmin'] = data['xmin'].astype(int)
data['ymin'] = data['ymin'].astype(int)
data['xmax'] = data['xmax'].astype(int)
data['ymax'] = data['ymax'].astype(int)

In [12]:
# img = cv2.imread('./data/' + info['image'])
# xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
# x = xmin
# y = ymin
# w = xmax - xmin
# h = ymax - ymin
# crop = img[y:y+h,x:x+w]
# cv2.imwrite('./crop.png',crop)
# cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)
# cv2.imwrite('./box.png',img)

TypeError: slice indices must be integers or None or have an __index__ method

In [13]:
def load_data():
    records = []
    for i in tqdm(range(len(data))):
        record = {}
        info = data.iloc[i]
        path = './data/' + info['image']
        record['file_name'] = path
        record['height'],record['width'] = cv2.imread(path).shape[:2]
        record['image_id'] = i
        record['annotations'] = [
            {
                'bbox':[info['xmin'],info['ymin'],info['xmax'],info['ymax']],
                'bbox_mode':BoxMode.XYXY_ABS,
                'category_id':0
            }
        ]
        records.append(record)
    np.save('./data.npy',records)
    return records

In [14]:
labels = ['car']

In [15]:
DatasetCatalog.register('data',lambda : load_data())
MetadataCatalog.get('data').set(thing_classes=labels)
metadata = MetadataCatalog.get('data')

In [16]:
model = "COCO-Detection/faster_rcnn_R_50_C4_3x.yaml"

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model))
cfg.DATASETS.TRAIN = ('data')
cfg.DATASETS.TEST = ()
cfg.SOLVER.STEPS = []
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(labels)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
predictor = DefaultPredictor(cfg)
cfg.MODEL.WEIGHTS = './output/model_final.pth'
evaluator = COCOEvaluator('data',output_dir='./output/')
val_loader = build_detection_test_loader(cfg,'data')
metrics = inference_on_dataset(predictor.model,val_loader,evaluator)
wandb.log(metrics)
img = cv2.imread('./data/vid_4_12220.jpg')
preds = predictor(img)
v = Visualizer(img[:,:,::-1],metadata=metadata)
v = v.draw_instance_predictions(preds['instances'].to('cpu'))
v = v.get_image()[:,:,::-1]
plt.figure(figsize=(12,6))
plt.imshow(v)
plt.savefig('./img.png')
plt.close()
wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()

wandb: wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-10-26 23:51:16.922852: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


[10/26 23:51:22 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

100%|████████████████████████████████████████| 559/559 [00:01<00:00, 324.22it/s]

[10/26 23:51:24 d2.data.build]: Removed 0 images with no usable annotations. 559 images left.
[10/26 23:51:24 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[10/26 23:51:24 d2.data.build]: Using training sampler TrainingSampler
[10/26 23:51:24 d2.data.common]: Serializing 559 elements to byte tensors and concatenating them all ...
[10/26 23:51:24 d2.data.common]: Serialized dataset takes 0.19 MiB



2021-10-26 23:51:24.605213: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 2048) in the checkpoint but (2, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 2048) in the checkpoint but (4, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model 

[10/26 23:51:27 d2.engine.train_loop]: Starting training from iteration 0
[10/26 23:51:40 d2.utils.events]:  eta: 0:02:45  iter: 19  total_loss: 1.38  loss_cls: 0.9006  loss_box_reg: 0.4518  loss_rpn_cls: 0.008332  loss_rpn_loc: 0.004952  time: 0.4690  data_time: 0.1661  lr: 9.7405e-06  max_mem: 2284M
[10/26 23:52:53 d2.utils.events]:  eta: 0:02:50  iter: 39  total_loss: 1.165  loss_cls: 0.6992  loss_box_reg: 0.4471  loss_rpn_cls: 0.007972  loss_rpn_loc: 0.004278  time: 2.1557  data_time: 0.0148  lr: 1.9731e-05  max_mem: 2284M
[10/26 23:53:00 d2.utils.events]:  eta: 0:02:40  iter: 59  total_loss: 0.8438  loss_cls: 0.4178  loss_box_reg: 0.3987  loss_rpn_cls: 0.006206  loss_rpn_loc: 0.004933  time: 1.5371  data_time: 0.0027  lr: 2.972e-05  max_mem: 2284M
[10/26 23:53:08 d2.utils.events]:  eta: 0:02:32  iter: 79  total_loss: 0.8069  loss_cls: 0.2955  loss_box_reg: 0.4882  loss_rpn_cls: 0.006152  loss_rpn_loc: 0.005304  time: 1.2336  data_time: 0.0027  lr: 3.9711e-05  max_mem: 2284M
[10/26

In [ ]:
torch.save(cfg,'cfg.pt')
torch.save(cfg,'cfg.pth')
torch.save(predictor,'predictor.pt')
torch.save(predictor,'predictor.pth')
torch.save(evaluator,'evaluator.pt')
torch.save(evaluator,'evaluator.pth')
torch.save(v,'img.pt')
torch.save(v,'img.pth')
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(labels,'labels.pt')
torch.save(labels,'labels.pth')
torch.save(metrics,'metrics.pt')
torch.save(metrics,'metrics.pth')